<a href="https://colab.research.google.com/github/hasune613/hello-world/blob/main/%E6%97%A5%E6%9C%AC%E8%AA%9E%E7%89%88BERT%E3%81%A7livedoor%E3%83%8B%E3%83%A5%E3%83%BC%E3%82%B9%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2021-09-09 00:28:40--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  5.51MB/s    in 1.5s    

2021-09-09 00:28:42 (5.51 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [ ]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)

['CHANGES.txt', 'peachy', 'smax', 'movie-enter', 'it-life-hack', 'sports-watch', 'dokujo-tsushin', 'README.txt', 'kaden-channel', 'livedoor-homme', 'topic-news']
カテゴリー数: 9
['peachy', 'smax', 'movie-enter', 'it-life-hack', 'sports-watch', 'dokujo-tsushin', 'kaden-channel', 'livedoor-homme', 'topic-news']


In [ ]:
# !ls ./data/livedoor/text/peachy

In [ ]:
#  ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある
   

0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



In [ ]:
## 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text
   

In [ ]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)

In [ ]:
## 0番目の文章とラベルを確認
print(list_text[0])
print(list_label[0])

「仕事」「プライベート」「恋愛」・・・がんばってる女の子ほど悩みの種やストレスの元は、多いもの。ストレスがたまると、カサカサ、ポツポツ・・・なんてことも。「ストレス」と「肌」の関係は、とても深く、密接なものです。毎日の暮らしのわずかな変化に、知らず知らずのうちに肌も不安定になってしまいます。「何かにとりつかれたように、最低な一日を過ごしたことがあります。」そう話してくれたのは、仕事のできるステキ女子を目指すA子さん（会社員・29歳）。「仕事で取引先と問題が発生。上司にこっぴどく叱られ、対応に追われなかなか帰れず、彼との約束の時間に遅れたら、彼がお怒りモード。自分もイライラしてしまって、結局はケンカしてデートにならず・・・。がんばってるつもりが空回り。一体何やってるんだろうと思いながらの帰りの電車、今度は酔っ払いに絡まれ、怖い思いをしたことがあります。もうそんな夜は、何も気力が残らず化粧を落とすだけで精一杯。ベッドへ直行して、眠るだけ。ひたすら癒されたい・・・と思いながら。（笑）」マンガやドラマみたいですが、誰もがなんで！？というぐらいに最低最悪な一日を過ごしたことはあるのではないでしょうか。そんなストレスがたまる一日もあなたの言葉や態度の一つで、回避できることもあるかもしれません。大人の女子力を身につけて一歩上のステキ女子を目指してはいかがでしょうか。まずは、あなたの女子力を診断！「大人の女子力診断」はコチラからそれでもやっぱりがんばる女の子には、肌も心も癒す時間が必要。夜寝る前にお肌のケアをする瞬間、お風呂でくつろぐ瞬間、恋人との電話など・・・皆さんの癒しのひとときは、何ですか？肌も心も傷ついた女性を癒す“ビューネくん”のCMでおなじみの「薬用 ビューネ」は、ストレスで不安定になりがちな肌をやさしく癒してくれます。大人の肌はとっても繊細。「薬用 ビューネ」は、「温泉」、「海」、「植物」の３つの自然のチカラを活かしたＳＰＡエッセンスを配合することで、肌荒れ・ニキビになりやすい肌のコンディションを整えて、ストレスに負けないすこやかなスベスベ肌へと導きます。ツイてる日もツイてない日も「ビューネ」で肌を癒しませんか？「Peachy」では、「“ビューネくん”に癒されたくなる瞬間」をTwitterで募集。ツイートいただいた方の中から抽選で5名様に「薬用 ビューネ」をプレゼント

In [ ]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()

(7376, 2)


,text,label
0,「仕事」「プライベート」「恋愛」・・・がんばってる女の子ほど悩みの種やストレスの元は、多いも...,peachy
1,厳しい夏もようやく終わりに近づいてきましたが、猛暑や紫外線によるダメージがたまり、身体やお肌...,peachy
2,「占い」は、いつだって女子の強い味方。雑誌やネットで新しい「○○占い」を見かけるたびに、試し...,peachy
3,男心を捉えるには“胃袋を掴むこと”は、昔から言われている必殺方法。確かに、周りの男子から聞く...,peachy
4,みなさんは、小さな頃からきちんと歯を磨いていましたか？ 面倒でイヤイヤやっていた人も多いので...,peachy


In [ ]:


# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()



{0: 'peachy', 1: 'smax', 2: 'movie-enter', 3: 'it-life-hack', 4: 'sports-watch', 5: 'dokujo-tsushin', 6: 'kaden-channel', 7: 'livedoor-homme', 8: 'topic-news'}
{'peachy': 0, 'smax': 1, 'movie-enter': 2, 'it-life-hack': 3, 'sports-watch': 4, 'dokujo-tsushin': 5, 'kaden-channel': 6, 'livedoor-homme': 7, 'topic-news': 8}


,text,label_index
0,「仕事」「プライベート」「恋愛」・・・がんばってる女の子ほど悩みの種やストレスの元は、多いも...,0
1,厳しい夏もようやく終わりに近づいてきましたが、猛暑や紫外線によるダメージがたまり、身体やお肌...,0
2,「占い」は、いつだって女子の強い味方。雑誌やネットで新しい「○○占い」を見かけるたびに、試し...,0
3,男心を捉えるには“胃袋を掴むこと”は、昔から言われている必殺方法。確かに、周りの男子から聞く...,0
4,みなさんは、小さな頃からきちんと歯を磨いていましたか？ 面倒でイヤイヤやっていた人も多いので...,0


In [ ]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

,text,label_index
0,“幸せ”って、何だろう？30代になり、子供の頃からの友達はほとんど結婚、子供を産み、専業主婦...,5
1,2月も既に後半。北日本ではまだ大雪が続いているが、沖縄では25度と早くも夏日になったとか。花...,6
2,先週前半は、嵐の前の静けさといった感じで、かなり平穏でした。それもあってニュースのネタ探しに...,3
3,2012年の幕開けと同時に恒例の冬のセールもスタート。早速セールに出かけて「昨年から狙ってい...,5
4,12日放送、日本テレビ「NEWS ZERO」では、「ドイツ香川を独占直撃 移籍報道の真相は」...,4


In [ ]:
dic_id2cat


{0: 'peachy',
 1: 'smax',
 2: 'movie-enter',
 3: 'it-life-hack',
 4: 'sports-watch',
 5: 'dokujo-tsushin',
 6: 'kaden-channel',
 7: 'livedoor-homme',
 8: 'topic-news'}

In [ ]:
dic_cat2id

{'dokujo-tsushin': 5,
 'it-life-hack': 3,
 'kaden-channel': 6,
 'livedoor-homme': 7,
 'movie-enter': 2,
 'peachy': 0,
 'smax': 1,
 'sports-watch': 4,
 'topic-news': 8}

In [ ]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)

(1475, 2)
(5901, 2)


In [ ]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")

In [ ]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make surl xz-utils file -y
!pip install mecab-python3==0.996.5

!pip install transformers==2.9.0

In [ ]:
! pip install -q unidic-lite

     |████████████████████████████████| 47.4 MB 39 kB/s 


In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

import torchtext#  torchtextを使用
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking')

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

In [ ]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義
max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL =data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定


In [ ]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = data.TabularDataset.splits(
    path='./', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])

In [ ]:
dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901)

In [ ]:
print(dataset_train.__len__())#4426
print(dataset_eval.__len__())#1475
print(dataset_test.__len__())#1475

4426
1475
1475


In [ ]:
item = next(iter(dataset_train))
print(item.Text)
print('長さ : ',len(item.Text) )
print('ラベル：',item.Label)

tensor([    2,  8198,   324,  4617,   768,  2121,  2481,    95,    12,     9,
            6,  6381,  3805,  5559, 28557,  7837,    93, 28472, 26003,  3012,
        12065,     7,  3104,    12,  2220,     6,   285,   176, 20484,    11,
         5625,    10, 21584,  1552, 14025,     8,  3325,     5,  1420,     9,
           59,  6262,    11,  2203,  1903,  1964,  2203,  8192,  1964,    13,
         6666,     6,  4818,    13,    15,    10,     8,  3712,    36, 10848,
        28472,  1710,  3964,    38,    23,   965,    32,   333,   155,    24,
           12,     9,     6,   486, 19398,     5,  5017,  2707,     6, 14758,
         4942,    60,    14,   608,     5, 20458,    11,  1449,    10,     8,
           36,  6122,     9,   151, 29217,  1552, 14025,    14,   109,   308,
           10,     8,  6788, 20252,     5,    80,  1738,  2764, 26985,    10,
         6172,  9366,  3415,     8, 17957,     6,  7477,   687,   918,     7,
         3083,  3051, 28547,   312,     6,  7913,     9, 22752, 

In [ ]:
# datasetの中身を文章に戻して確認
print(tokenizer.convert_ids_to_tokens(item.Text.tolist()))#文章
dic_id2cat[int(item.Label)]#id


['[CLS]', 'WBC', '世界', 'フライ', '級', '王座', '統一', '戦', 'で', 'は', '、', '暫定', '王者', 'ポン', '##サ', '##クレ', '##ック', '##・', '##ウォン', '##ジョン', '##カム', 'に', '判定', 'で', '敗れ', '、', 'プロ', '初', '黒星', 'を', '喫し', 'た', '亀田', '興', '毅', '。', '周囲', 'の', '評価', 'は', 'その', '敗戦', 'を', '“', '経験', '”', '“', '勉強', '”', 'と', '語り', '、', 'よし', 'と', 'し', 'た', '。', 'TBS', '「', 'サンデー', '##・', '##ジャ', '##ポン', '」', '(', '28', '日', '放送', '分', ')', 'で', 'は', '、', '番組', 'コメンテーター', 'の', '高橋', 'ジョージ', '、', 'テリー', '伊藤', 'ら', 'が', '試合', 'の', '感想', 'を', '述べ', 'た', '。', '「', 'スピード', 'は', '全', '##然', '興', '毅', 'が', '上', 'だっ', 'た', '。', '引っ', '##掛かり', 'の', 'ない', 'まま', '終わっ', 'ちゃっ', 'た', 'って', '##いう', '感じ', '。', 'だから', '、', '結論', 'だけ', '最初', 'に', '言っ', 'ちゃ', '##え', 'ば', '、', '俺', 'は', 'よかっ', 'た', 'と', '思う', '。', '変', 'な', '話', 'です', 'けど', '、', 'いずれ', 'は', 'どんな', 'ビッグ', 'な', 'ファイター', 'で', 'も', '一', '敗', 'は', 'し', 'ます', 'よ', '。', 'だから', '、', 'この', '時', 'に', '負け', 'て', 'おい', 'て', '、', 'すぐ', '立ち', '##直', '##っ', 'て', 'やっ', 'て', 'ほしい', '。

'sports-watch'

In [ ]:
# DataLoader作成（torchtextの文脈⇒⇒iterater）

batch_size = 16 # 16,32推奨

dl_train =data.Iterator(
    dataset_train, batch_size = batch_size,train =True)

dl_eval = data.Iterator(
    dataset_eval, batch_size = batch_size , train = False, sort = False)

dl_test = data.Iterator(
    dataset_test, batch_size = batch_size , train = False, sort = False)


# 辞書オブジェクトにまとめる
dataloaders_dict = {'train':dl_train, 'val':dl_eval}

In [ ]:
batch = next(iter(dl_test))
print(batch)
print(batch.Text[0].shape)
print(batch.Label.shape)



[torchtext.legacy.data.batch.Batch of size 16]
	[.Text]:('[torch.LongTensor of size 16x512]', '[torch.LongTensor of size 16]')
	[.Label]:[torch.LongTensor of size 16]
torch.Size([16, 512])
torch.Size([16])


In [ ]:
from transformers.modeling_bert import BertModel

In [ ]:
# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# このベースの日本語BERTモデルに、入力が768次元、出力が9クラスの全結合層を用意。
# 順伝搬のforward関数を用意します。

from torch import nn

class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''
    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model# 日本語学習済みのBERTモデル
        # headにクラス予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=768, out_features=9)
        # 重み初期化処理
        nn.init.normal_(self.cls.weight,std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self,input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''
        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる

        result = self.bert(input_ids)# reult は、sequence_output, pooled_output
        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]# 最初の0がsequence_outputを示す
        vec_0 = vec_0[:,0,:]# 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)# sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0) # 全結合層

        return output

In [ ]:
# モデル構築
net = BertForLivedoor()

#訓練用モードに設定
net.train()

print('ネットワーク設定完了')

ネットワーク設定完了


In [ ]:
# ファインチューニング
# BERTのself-Attentionの繰り返し12層のうちの、最後の層のみをパラメータ学習させる設定

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad=False
# 2. BertLayerモジュールの最後の層のみ勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True


In [ ]:
# 最適化関数(Adam)
import torch.optim as optim

# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
        {'params':net.bert.encoder.layer[-1].parameters(),'lr':5e-5},
        {'params':net.cls.parameters(),'lr':1e-4}
])
# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

In [ ]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net

In [ ]:
# num_epochs = 1
# net_trained = train_model(net, dataloaders_dict,
#                           criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 0.7048 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 20 || Loss: 1.1246 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 30 || Loss: 0.4805 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 40 || Loss: 0.5414 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 50 || Loss: 0.5317 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 60 || Loss: 0.6892 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 70 || Loss: 0.6289 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 80 || Loss: 0.5640 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 90 || Loss: 0.8566 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 100 || Loss: 0.3890 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 110 || Loss: 0.5582 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 120 || Loss: 0.4103 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 130 || Loss: 0.6775 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 140 || Loss: 0.5574 || 10iter. || 本イテレーションの正解率：0.9375
イテレーション 150 || Loss: 0.5359 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 160 || Loss: 0.7830 || 10iter. || 本イテレーションの正解率：0.75

In [ ]:
from tqdm import tqdm

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net_trained.eval()
net_trained.to(device)

epoch_corrects = 0

for batch in tqdm(dl_test):
    inputs = batch.Text[0].to(device)
    labels = batch.Label.to(device)

    with torch.set_grad_enabled(False):

        outputs = net_trained(inputs)

        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        epoch_corrects += torch.sum(preds == labels.data)

epoch_acc = epoch_corrects.double() / len(dl_test.dataset)

print(f'テストデータ{len(dl_test.dataset)}個での正解率 : {epoch_acc:.4f}')

100%|██████████| 93/93 [01:38<00:00,  1.06s/it]


テストデータ1475個での正解率 : 0.8841
